In [167]:
#### specify path of Julia Code for 2D Models ####
### By default the working directory ./2D_Models ###
path = string( pwd(), "/2D_Models/")

"/home/jakob/Dokumente/Repositories/Modeling_Framework/2D_Models/"

In [178]:
##### include 2D Modelling Framework ####
include(string(path, "./2D_model_library.jl"));


########## OPERATIONAL ##########
##### how to chose the actual velocity? #####

#op_model = "Constant"
#op_model = "Collision_Free_Speed"
#op_model = "RVO"
#op_model = "SocialForce"
#op_model = "AV_Model"
op_model = "AV_IN_Model"
#op_model = "PowerLaw"
#op_model = "IN_Costfunction"
#op_model = "IN_Model"

include_op_model(op_model, path)

########## TACTICAL ##########
#### how to chose the preferred velocity? ###

tact_model = "Constant"
#tact_model = "IN_Model"

include_tact_model(tact_model, path)


########## STRATEGIC ##########
### how to chose the desired velocity ###

strat_model = "Constant"
#strat_model = "Towards_Goal"

include_strat_model(strat_model, path)


########## OPTIMIZATION SCHEME ##########
### If optimization of a cost-function is included, how to solve? ###

#optimization_scheme = "Regular_Sampling"
optimization_scheme = "Random_Sampling"

include_optimization_scheme(optimization_scheme, path)


##### which update scheme? #####

update = "Parallel_Update"
#update = "Step_Update"

include_update(update, path)


##### which order? i.e. Acceleration or Velocity based? ####
#order = "first"
order = "second"

include_order(order, path)


##### PERCEPTIONAL MODEL ####
perception_model = "None"
#perception_model = "Only_in_Front"

include_perception_model(perception_model, path)

In [179]:
#### Special Scenario? ####
x = "Obstacle_Intruder"
#x = "Diminish_ANT_LF"

load_scenario(x, path)

In [180]:
#### parameters of the agents ####
p = [1.0, 0.0, 2.0, 1.0, 0.3, 0.0, 0.0, 0.1, 5.0, 15.0, 0.0, 0.0, 0.5, 0.0]
# ModelParameter: v_max, v_des, T, T2, l, step_time, τ_A, τ_R, α, β, ζ_h, ζ_v, r, ϕ
p_desc = "v_max, v_des, T, T2, l, step_time, τ_A, τ_R, α, β, ζ_h, ζ_v, r, ϕ"

### of the system ###
ρ = 2.0
system_size = (4.0, 4.0) #important for periodic boundaries
N = Int(round(ρ*system_size[1]*system_size[2]))

### of the simulation ###
sim_p = [0.0, 10.0+0.0, 0.05, 0.01, 3.0] #t_relax, t_max, dt_save, dt, r
sim_p_desc = "t_relax, t_max, dt_save, dt, r"

### initialize geometry ###
geometrie = create_geometry_single_obstacle((37.5,31.5), 0.3)

### initialize crowd ###
menge = create_crowd(N, geometrie)

Init_Hom_Parameters!(p, menge)
Init_Hom_Vels!(menge, 1.0)
Init_E_des!(menge, (1.0, 0.0))

initialize_random_positions(system_size, menge, geometrie)
Init_Random_Headings!(menge)

vel, l_obs = 0.3, 0.7
Init_Nth_Agent_as_Moving_Obstacle!(menge, system_size, l_obs, vel);

In [181]:
positions, headings = Simulate!(menge, geometrie, sim_p[1], sim_p[2], sim_p[3], sim_p[4], sim_p[5], system_size);

In [182]:
using GR
GR.inline("mov")

j = 1

dt_save = sim_p[3]
geometry_x, geometry_y = rewrite_geometry(geometrie)
boundaries_x, boundaries_y = boundaries_rectangle_positions(system_size)

while j*sim_p[3] < sim_p[2]-sim_p[1]
    
        GR.clearws()

        GR.setwindow(-1, 11, -1, 11)
        GR.setviewport(0.0, 1, 0.0, 1)
    

        plot_boundaries(boundaries_x, boundaries_y, 0.2, 999)        
        plot_geometry(geometry_x, geometry_y, 3.7, 20)

        plot_agents(positions[j, 1:N-1], headings[j, 1:N-1], 2.3, 100, 0.7, 999, 0.1)
    
        #plot moving obstacle
        plot_agents(positions[j, N:end], headings[j, N:end], 3.7, 97, 0.0, 999, 0.1)

        model = op_model
        timer = string("time ", round(j*dt_save, digits = 1), "s")
        GR.text(0.7, 0.95, timer)
        GR.text(0.7, 0.9, model)
        
        GR.updatews()
    
        j = j+1
end

In [183]:
GR.show()

GR.HTML("<video autoplay controls><source type=\"video/mp4\" src=\"data:video/mp4;base64,AAAAFGZ0eXBxdCAgAAACAHF0ICAAAAAId2lkZQAXmmZtZGF0AAACsQYF//+t3EXpvebZSLeWLNgg2SPu73gyNjQgLSBjb3JlIDE2MyAtIEguMjY0L01QRUctNCBBVkMgY29kZWMgLSBDb3B5bGVmdCAyMDAzLTIwMjEgLSBodHRwOi8vd3d3LnZpZGVvbGFuLm9yZy94MjY0Lmh0bWwgLSBvcHRpb25zOiBjYWJhYz0xIHJlZj0zIGRlYmxvY2s9MTowOjAgYW5hbHlzZT0weDM6MHgxMTMgbWU9aGV4IHN1Ym1lPTcgcHN5PTEgcHN5X3JkPTEuMDA6MC4wMCBtaXhlZF9yZWY9MSBtZV9yYW5nZT0xNiBjaHJvbWFfbWU9MSB0cmVsbGlzPTEgOHg4ZGN0PTEgY3FtPTAgZGVhZHpvbmU9MjEsMTEgZmFzdF9wc2tpcD0xIGNocm9tYV9xcF9vZmZzZXQ9LTIgdGhyZWFkcz0xMiBsb29rYWhlYWRfdGhyZWFkcz0yIHNsaWNlZF90aHJlYWRzPTAgbnI9MCBkZWNpbWF0ZT0xIGludGVybGFjZWQ9MCBibHVyYXlfY29tcGF0PTAgY29uc3RyYWluZWRfaW50cmE9MCBiZnJhbWVzPTMgYl9weXJhbWlkPTIgYl9hZGFwdD0xIGJfYmlhcz0wIGRpcmVjdD0xIHdlaWdodGI9MSBvcGVuX2dvcD0wIHdlaWdodHA9MiBrZXlpbnQ9MjUwIGtleWludF9taW49MjQgc2NlbmVjdXQ9NDAgaW50cmFfcmVmcmVzaD0wIHJjX2xvb2thaGVhZD00MCByYz1hYnIgbWJ0cmVlPTEgYml0cmF0ZT00MDAwIHJhdGV0b2w9MS4wIHFjb21wPTAuNjAgcXBtaW49MCBxcG1heD02OSBxcHN0ZXA9NCBpcF9yYXRpbz0xLjQwIGFxPTE6MS4wMACAAAA7P2WIhAAa//731LfMsu1Tb7YLqPeiniZfQ3UlAZuWxO06gAAAAwAAAwAAAwDpVIJjfO4A0Yy0AAGmi/SZxwabwAHQGDGM77Gdn2nqvAAWvZ6FoOJ+m2zddvNGY1PNZxqSA/qFRZmVjZ76nsQkA36vZ1t90oJ7I5RD2VckVMfyHmVJdrl7Q6j0N37upVso8tOotrXFVTjVGkhhoXrdZHzshRDEePDLsoPQjxXXWdPdzOhTyL0NMFdep/kRldWxURL+4zu5YHAGRsBRQOfxkgwNt2cZ/Q8WW5KWwTOAAwkeRIsc/4qZZiyzARb2d7Dcj2Abc6I03wQ9K20p1mJ0tpttY3GecW1+pahjzbhE60xtX/7oWToJUJPpqnUQS9VJMcEm3MrtcS95kjduVGYe3cBjsQ7muzjdxvyWTP0qeSY5cODrNsmXb85rPfYBi1qrP8SGvDYBhnt679EvrOJ2ldPdYo2kLXa/Afa8X8n3X0sl4ckDoXDM8tyxBuqTCR8EcKTjGUMZpXEpkPJ5RJ/V1Oh/mxMrEB3wsiXzlpmChXFtG3hzIcydZtGJE26p4/////42sUJlo2xGuiCjhFiTK+luztOld9Qc/jpud47dZNQJRTpOxYB4XTGcA+w0ZSfH+sajowD05ecqF0sY4YXz6sMzVXmvR96OQVvOEIAcExeL6+37+9WOTu+neRCr0s5MVO6CN8WpnZR1ZqT4Dqwp4LBvO9c5HTWyz6n3y1fpPxPKDl54s4P/I2x3g6qV9xmLELUAQqro6nBV706LIk6OfunvsFTnfE6mQGH72Rf7xiGXfWP7yoQzTCmpKZSUcBcUGsYo7vZSjYAyr0AVr5vB7AHT9/6SXmqR/m0U5ni3I0GgF5ZF6JaqGHp261SY08qVf/ygp0LdMBPZR5DnDUVj4gXhTapFizvLPCrukBlZPCD16WcpOkGB/oxY9YXT5+gErvwwmtneBNMTtydFW0YuoAbSTguTbVp2ghWrO5a5yL1jShsy5Qlo+uInOflOlZyeI/jNuuBG+OcmzOMk1I+pNXJgwDOmoTllFSpkItpAAhsVF/vv2YR/tfgZDvVqXKfxTat3QAB4G0MKdNXEj3zeVNoiFR5nIE1+vY3spL3hAa1JeaFoq6GwqXaZYkFQq+xdZLrsnTEVNf4xwe78zjc+4zQzmkIxIqhsax3atjxmistcmm0a/7XFxMb+PNLSK3NkDYyvS8ykZ3xewoS5owNSpxEz9jRM1vMNg2IwW8pWL0qOJTdITUZwK7T39/Sy3Lk4qFt7cFkUeINLdA0Fnxv3+KMoTjwpYVxCVGqXo1XUNQcan3i0Gd/pCdjeVYEzQuUiaw/EbozlC5fAePIzzqkuYztkuVdTKzds2Dy5qIcorEehTXFE2a9i3RMhmxmAE7RV7tTJfS+HjFDAjeKhVt+7h51yabBbYwli3pMgVnwcuWs6IIQZdD7bN+5VjfXAn0hP1WINnDLVh+cLNaRMbPee4dqBRzjyF7yqoGYle0HV4/olcQIsmEOQF1eu0oKJqj0IVK2tRbGTEgPGXPo4bZY0mmYmwAtpXt36jad+wFj79cRAD5Ovgrnqwl5QfeqbeXlteClutgL9+ezCRjzmfuMGz96+F/Xj0LPSt84Z4MMjESCS4ergfH0il14ntvT8XXr/n3AA2kXQWzVNj6d/hLS7m/W49xGapc2APtINnP/+C3N6J3EO9AQSSzjYdd8f5bb4BjEaF3ymQlhtabRt+6egTvC+EAC0gGoMF+OzJZF0u35lB+NKac4RXfs2AmQDEpxEB/4tO4x/Hn9PbC3z/NngLg4UOtXpqpTJoL6wkNjP4NDuA67bw2xKnXPCX3zhLXkveZXEz8rNfn288+Vd5c7ew8iW9RmfuEXE3zCvXubqXuehOcAnFcKDYTbDvU0DHq2KsHUC5Mzu2yjXKiI+FGACeGzQImv+X7P/hcOYT90OzUXmv06cpOfRlvQ/ce6lX9/6iIpC6dJP8ZnmUvwDv4lxH9h0MtLMTDA64FA10VGVbyHKVsZDvJKTiJ6UCqKEOkWPLqO88rLVSiqem1+BLKRf6h9z8oXdCPOFbR4ZIyVRG9AaEXfi3D/UWwa+u8Mhd5q/pwxanviiHevFl/7vL1zJlxw080MwmVQwSgz8eZe6rpm+x28E48slLdDp2kWaHxceit5bZYapuxCVe0DsMVxDHHPcdRIq7tVLIcCSyVXY3XiDUU3rRnRUPFc9TGqVh+dQdLbRCuih9o7noF8S1bMchoPXmfJnHi1qNCIYRfDsXR+vINlrT17Or+xOUjKBMM+IjggqiZhPTjTFF2YheO23aW9xpXqNz/LvY0/TKC30IKQStkSooAiVXf3Jfq8wi1jOxFhpn0OIOVoHVEkaWJn9OtaSVKpjxIvG8e7vvzPb6maOi+pGynpkZnB25szdU+troF+HC39fFxkmqdTsqVfdwvhllzRo5am/ijwxVSzlBOP6HkAjcTk0ia3iGabRu5zHUVXJeC17prwCOU6DErY6KSd1NDduhe0Eh2MDf6B373TydomxZcLp5uw57P6gQhQRzo+g+A2uJ7oIm7LbBX7YWb7i+WPGcFcYba9jkh+/RxZd0ETWpTQFHGI9SAofYWsC1p0MiaVHT4r25bDDDXDE4thb/VdXXgGI/XlzyJ6zxUoqGdvroWiXQIGjXXm/r3Tw1/LVtB6Y+wmjQS4JOuvdRbrADvFmZLwbCuwtb+X/sYHnbP3j0C7CLthnXudnd1GyNietYJaDMrL+u5/mCtnvYWwt84gznsA8vBgi3c0bdqqNfLlSSzYkqSXWb6FHWkMnhT9jvoCX2d5dinTuDAdPxUnStJaYCn8WNrWOWFRvNXqFjvc8qYhhMaEaxtGmiIiSLUDC+ZDSpUMLj7+l6wgWMsfKur2mXw2NGC+enA32l9C/q79qQG43oZK7AUP

In [ ]:
Path = "/home/jakob/Dokumente/Sim_Data/"
dir = "Intruder_NO_ANT_4"
header = Header(p, p_desc, sim_p, sim_p_desc, op_model, tact_model, order, update, N, system_size);

Save_Data!(Path, dir, header, positions)

In [ ]:
samples = 20

for i in 1:samples
    
    Random.seed!()
    
    #if mod(i, 10) == 0
        println(i)
    #end
    
    Init_Hom_Vels!(menge, 0.0)
    Init_Rand_Rect!(a0, a, b0, b, menge, geometrie)
    initialize_random_headings(menge)

    vel, l_obs = 0.3, 0.7

    Init_Nth_Agent_as_Moving_Obstacle!(menge, system_size, l_obs, vel);
    
    positions, headings, ttcs = Simulate!(menge, geometrie, sim_p[1], sim_p[2], sim_p[3], sim_p[4], sim_p[5], system_size);
    Save_Data!(Path, dir, header, positions)
    
end

In [131]:
### POWERLAW MODEL ###

f(x) = x^2*exp(-1/x)

function ϕ(a::agent, menge::crowd, geometrie::geometry, system_size)
    
    ϕ_ = 0.0
    
    for i in 2:a.neighbors_geometry[1]+1

        b = geometrie.element[a.neighbors_geometry[i]]

        ϕ_ += f(AV(a, b, system_size))

    end
    
    for i in 2:a.neighbors_agents[1]+1

        b = menge.agent[a.neighbors_agents[i]]

        ϕ_ += f(AV(a, b, system_size))

    end
    
    ϕ_
    
end

ϕ (generic function with 1 method)

In [122]:
function Intersect_Line_Circle(a::agent, b::agent, c::agent)

    r_i = a.pos[1]+im*a.pos[2]
    r_j = b.pos[1]+im*b.pos[2]
    r_k = c.pos[1]+im*c.pos[2]

    z = (r_k - r_i )/(r_j - r_i)

    distance = 0.0 < real(z) < 1.0 ? abs(imag(z)*(r_j - r_i)) : min(abs(r_j - r_k), abs(r_i - r_k))

    if distance <= c.l/2
        true
    else
        false
    end
end


function Perceivable(a::agent, b::agent, menge::crowd)

    perceivable_ = true

    for c in menge.agent

        if c != a && c != b && Intersect_Line_Circle(a, b, c, system_size) == true

            perceivable_ = false
            break

        end
    end

    perceivable_

end

Perceivable (generic function with 5 methods)